In [ ]:
import sys
sys.path.append("..") # for sibling import

import torch
import walnut
import walnut.tensor_utils as tu
import walnut.nn as nn

In [ ]:
B, C_in, C_out = 10, 5, 1

X = walnut.randn((B, C_in))
W1 = walnut.randn((C_in, C_out))*10 # (K, C, Y, X)
W2 = walnut.randn((C_in, C_out+1))*10 # (K, C, Y, X)
W3 = walnut.randn((C_in, C_out+2))*10 # (K, C, Y, X)

t_x = torch.from_numpy(X.data).float()
t_x.requires_grad = True
t_w1 = torch.nn.Parameter(torch.from_numpy(W1.T).float(), requires_grad=True)
t_w2 = torch.nn.Parameter(torch.from_numpy(W2.T).float(), requires_grad=True)
t_w3 = torch.nn.Parameter(torch.from_numpy(W3.T).float(), requires_grad=True)

### Forward

In [ ]:
lin1 = nn.layers.Linear(C_in, C_out, weights=W1, use_bias=False)
lin2 = nn.layers.Linear(C_in, C_out, weights=W2, use_bias=False)
lin3 = nn.layers.Linear(C_in, C_out, weights=W3, use_bias=False)

c = nn.containers.ParallelContainer([lin1, lin2, lin3], -1)
c.training = True
c.keep_output = True

In [ ]:
out = c(X)
out

In [ ]:
t_lin1 = torch.nn.Linear(C_in, C_out, bias=False)
t_lin1.weight = t_w1
t_lin2 = torch.nn.Linear(C_in, C_out, bias=False)
t_lin2.weight = t_w2
t_lin3 = torch.nn.Linear(C_in, C_out, bias=False)
t_lin3.weight = t_w3

t_c = [t_lin1, t_lin2, t_lin3]
t_out = torch.cat([h(t_x) for h in t_c], -1)
t_out

In [ ]:
for m in c.sub_modules:
    print(m.y.shape)

In [ ]:
dy = tu.randn((out.shape)).data
t_dy = torch.from_numpy(dy).float()

x_grad = c.backward(dy)
t_out.backward(t_dy)

In [ ]:
x_grad

In [ ]:
t_x.grad